# Arbol de decision

In [ ]:
import graphviz
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, auc

import graphviz

In [ ]:
y = pd.read_csv('Datasets/tp-2020-2c-train-cols1.csv')

In [ ]:
X = pd.read_csv('Datasets/tp-2020-2c-train-cols2.csv')

Llamar a preprocessing para el arbol

In [ ]:
y.drop(columns = 'id_usuario',inplace = True)

In [ ]:
X.drop(columns = ['id_usuario','nombre','id_ticket','fila'],inplace= True)

In [ ]:
X["tipo_de_sala"] = X["tipo_de_sala"].astype("category")
X["genero"] = X["genero"].astype("category")
X["nombre_sede"] = X["nombre_sede"].astype("category")

In [ ]:
X["edad"].fillna(X["edad"].mean(), inplace = True)

In [ ]:
X.dropna(inplace = True)

In [ ]:
y = y.loc[X.index]

In [ ]:
X = pd.get_dummies(X, drop_first=True, columns=['tipo_de_sala', 'genero', 'nombre_sede'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Árboles de decisión

In [ ]:
def construir_arbol(X_train, y_train, profundidad_maxima, criterio):
    arbol_clasificador = tree.DecisionTreeClassifier(random_state=117, max_depth=profundidad_maxima, criterion = criterio)
    arbol_clasificador.fit(X_train, y_train)    
    return arbol_clasificador

In [ ]:
def predecir(X_test, arbol_clasificador):
    y_pred = arbol_clasificador.predict(X_test)
    return y_pred

In [ ]:
def accuracy(y_pred, y_test):
    accuracy_ = pd.Series(np.array(y_test["volveria"]) == y_pred).mean()
    return accuracy_

In [ ]:
def graficar_arbol(arbol_clasificador, profundidad_maxima):
    dot_data = tree.export_graphviz(arbol_clasificador, feature_names=X.columns.to_list(),  max_depth = profundidad_maxima) 
    graph = graphviz.Source(dot_data) 
    graph.render("arbol de decision")

In [ ]:
def obtener_accuracies_segun_profundidad(profundidades, criterio):
    accuracies = []
    profundidades_maximas = []

    for profundidad_maxima in profundidades:
        arbol_clasificador = construir_arbol(X_train, y_train, profundidad_maxima, criterio)
        y_pred = predecir(X_test, arbol_clasificador)
        accuracy_ = accuracy(y_pred, y_test)

        accuracies.append(accuracy_)
        profundidades_maximas.append(profundidad_maxima)
        
    return accuracies, profundidades_maximas

In [ ]:
def graficar_accuracy_vs_profundidad(accuracy, profundidades):
    plt.subplots(dpi = 120)
    plt.plot(profundidades, accuracies, color='lightblue', linestyle='dashed', linewidth = 2, 
             marker='o', markerfacecolor='blue', markersize=7) 
    plt.xlabel('Profundidad del arbol')
    plt.ylabel('Accuracy sobre train')
    plt.title("Accuracy segun profundidad")

    axes = plt.gca()
    axes.set_ylim([0,1])

### Accuracy segun profundidad

Lo primero que haremos es construir y entrenar 10 árboles con el criterio de impureza de Gini de profundidades 1 a 10 y luego otros 10 árboles con el criterio de ganancia de información.

Para cada uno de estos árboles calculamos el accuracy y vemos en que profundidad se alcanzó el máximo accuracy.

In [ ]:
accuracies, profundidades = obtener_accuracies_segun_profundidad([1,2,3,4,5,6,7,8,9,10], "gini")

In [ ]:
graficar_accuracy_vs_profundidad(accuracies, profundidades)

In [ ]:
accuracies, profundidades = obtener_accuracies_segun_profundidad([1,2,3,4,5,6,7,8,9,10], "entropy")

In [ ]:
graficar_accuracy_vs_profundidad(accuracies, profundidades)

Observamos que en ambos casos el máximo accuracy se alcanzó para la profundidad 4 y por eso utilizaremos esta profundidad para estudiar las matrices de confusión

In [ ]:
arbol_clasificador = construir_arbol(X_train, y_train, 4, "gini")
y_pred = predecir(X_test, arbol_clasificador)

In [ ]:
arbol_clasificador_ganancia_informacion = construir_arbol(X_train, y_train, 4, "entropy")
y_pred_ganancia_informacion = predecir(X_test, arbol_clasificador_ganancia_informacion)

# Matriz de confusión árbol de decisión criterio de impureza de Gini

In [ ]:
fig, ax = plt.subplots(dpi =150)   
sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, vmin = 0, yticklabels=["Volveria","No volveria"], xticklabels=["Volveria", "No Volveria"], ax=ax)
ax.set_title("Matriz de confusión del arbol de decisión (Gini)")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

In [ ]:
fig, ax = plt.subplots(dpi =150)   
sns.heatmap(confusion_matrix(y_test, y_pred_ganancia_informacion), annot = True, vmin = 0, yticklabels=["Volveria","No volveria"], xticklabels=["Volveria", "No Volveria"], ax=ax)
ax.set_title("Matriz de confusion del arbol de decision (Ganancia de Información)")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

Podemos ver que para la mejor profundidad, el árbol conseguido por el criterio de gini es mejor. Por lo tanto estudiaremos las metricas AUC-ROC para este árbol.

# Metricas

In [ ]:
print(classification_report(y_pred, y_test, target_names=["Volveria", "No Voleria"]))

In [ ]:
def graficarAUCROC(X_test, X_train, y_test, y_train):

    fpr_cnb_test,tpr_cnb_test,thresholds_cnb_test = roc_curve(y_test, arbol_clasificador.predict_proba(X_test)[:,1])
    fpr_cnb_train,tpr_cnb_train,thresholds_cnb_train = roc_curve(y_train, arbol_clasificador.predict_proba(X_train)[:,1])

    zero_test = np.argmin(np.abs(thresholds_cnb_test))
    zero_train = np.argmin(np.abs(thresholds_cnb_train))

    plt.plot(fpr_cnb_train,tpr_cnb_train,label="ROC Curve Decision Tree Classifier Train")
    plt.plot(fpr_cnb_test,tpr_cnb_test,label="ROC Curve Decision Tree Classifier Test")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.plot(fpr_cnb_test[zero_test],tpr_cnb_test[zero_test],'o',markersize=10,label="threshold zero test",fillstyle="none",c="k",mew=2)
    plt.plot(fpr_cnb_train[zero_train],tpr_cnb_train[zero_train],'x',markersize=10,label="threshold zero train",fillstyle="none",c="k",mew=2)

    plt.legend(loc=4)

In [ ]:
graficarAUCROC(X_test, X_train, y_test, y_train)

# Mejor profundidad por grid search AUC-ROC cross validation

Decidimos también buscar la mejor profundidad utilizando cross validation y la métrica AUC-ROC y compararla con la que se obtuvo anteriormente utilizando accuracy. Solo consideraremos árboles construidos el criterio de gini.

In [ ]:
def mejor_profundidad_segun_AUCROC_CV(X, y):

    mejor_auc_roc = 0
    mejor_profundidad = None

    for profundidad in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:

        kf = StratifiedKFold(n_splits=8)
        metricas = []

        for fold_idx, (train_index, test_index) in enumerate(kf.split(X, y)):

            arbol_clasificador = construir_arbol(X.iloc[train_index], y.iloc[train_index], profundidad, "gini")        
            y_pred = predecir(X.iloc[test_index], arbol_clasificador)
            auc_roc = roc_auc_score(y.iloc[test_index], y_pred)  
            metricas.append(auc_roc)

        if np.mean(metricas) >= mejor_auc_roc:
            mejor_auc_roc = np.mean(metricas)
            mejor_profundidad = profundidad
            
    return mejor_profundidad, mejor_auc_roc

In [ ]:
mejor_profundidad, mejor_auc_roc = mejor_profundidad_segun_AUCROC_CV(X, y)

In [ ]:
mejor_profundidad

Segun esta metrica y para las profundidades de 1 a 10 la mejor profundidad es 6. Sin embargo nosotros consideramos que es mejor la profundidad obtenida con la metrica de accuracy, que fue 4. Esto es por que al ser una profundidad menor es mas interpretable, ademas la metrica de accuracy es mas interpretable que la de AUC_ROC.